In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
# имена папок. они же - имена классов.
labels=['karabin','hryvnia','stepler','trash']
#извлекаем списки файлов для каждой папки.
filelists={}
for i in labels:
    filelists[i]=os.listdir(i)

# Сейчас надо схлопнуть один большой дескриптор (nx128 для SIFT) из всех изображений.

In [3]:
orb=cv2.ORB_create()
sift=cv2.SIFT_create(nfeatures=500)
brisk=cv2.BRISK_create()

estimator=sift
name_estimator="sift"

#SIFT очень долго выполняет это место(~30 минут.)
des_list=[]
des_dict={}
failed_files=[] #файлы, для которых des=None
for i in labels:
    for j in filelists[i]:
        pathname=f'{i}/{j}'
        img=cv2.imread(pathname,0)
        img=cv2.resize( img, (img.shape[0]//4,img.shape[1]//4) )
        kp,des=estimator.detectAndCompute(img,None)
        if(des is None):
            failed_files.append(pathname)
            continue
        des=des.astype('float32')
        des_list.append(des)
        des_dict[pathname]=des
        print(pathname, des.shape)

karabin/(69).jpg (242, 128)
karabin/(73).jpg (500, 128)
karabin/(60).jpg (501, 128)
karabin/(23).jpg (190, 128)
karabin/(67).jpg (500, 128)
karabin/(25).jpg (500, 128)
karabin/(71).jpg (500, 128)
karabin/(82).jpg (500, 128)
karabin/(38).jpg (500, 128)
karabin/(1).jpg (500, 128)
karabin/(79).jpg (500, 128)
karabin/(40).jpg (500, 128)
karabin/(4).jpg (500, 128)
karabin/(63).jpg (500, 128)
karabin/(64).jpg (501, 128)
karabin/(76).jpg (500, 128)
karabin/(39).jpg (500, 128)
karabin/(83).jpg (500, 128)
karabin/(95).jpg (500, 128)
karabin/(34).jpg (500, 128)
karabin/(29).jpg (37, 128)
karabin/(12).jpg (500, 128)
karabin/(17).jpg (500, 128)
karabin/(14).jpg (500, 128)
karabin/(48).jpg (500, 128)
karabin/(35).jpg (500, 128)
karabin/(45).jpg (500, 128)
karabin/(5).jpg (359, 128)
karabin/(74).jpg (500, 128)
karabin/(18).jpg (501, 128)
karabin/(87).jpg (500, 128)
karabin/(51).jpg (500, 128)
karabin/(56).jpg (500, 128)
karabin/(19).jpg (502, 128)
karabin/(81).jpg (295, 128)
karabin/(99).jpg (500, 1

stepler/(40).jpg (285, 128)
stepler/(4).jpg (163, 128)
stepler/(63).jpg (233, 128)
stepler/(64).jpg (325, 128)
stepler/(76).jpg (296, 128)
stepler/(39).jpg (245, 128)
stepler/KIMG1092.jpg (500, 128)
stepler/(83).jpg (500, 128)
stepler/KIMG1113.jpg (346, 128)
stepler/(95).jpg (438, 128)
stepler/KIMG1089.jpg (500, 128)
stepler/(34).jpg (109, 128)
stepler/(29).jpg (340, 128)
stepler/(12).jpg (500, 128)
stepler/KIMG1122.jpg (501, 128)
stepler/(17).jpg (501, 128)
stepler/(14).jpg (500, 128)
stepler/(48).jpg (500, 128)
stepler/(35).jpg (61, 128)
stepler/(45).jpg (205, 128)
stepler/KIMG1125.jpg (500, 128)
stepler/KIMG1101.jpg (500, 128)
stepler/(5).jpg (82, 128)
stepler/KIMG1094.jpg (500, 128)
stepler/(74).jpg (482, 128)
stepler/KIMG1093.jpg (500, 128)
stepler/(18).jpg (500, 128)
stepler/(87).jpg (379, 128)
stepler/(51).jpg (500, 128)
stepler/(56).jpg (501, 128)
stepler/(19).jpg (500, 128)
stepler/(81).jpg (501, 128)
stepler/(99).jpg (500, 128)
stepler/(20).jpg (500, 128)
stepler/(15).jpg (50

In [4]:
#тяжелая ячейка, комп начнет гудеть
from sklearn.cluster import KMeans
X=np.vstack(des_list)
kmeans=KMeans(n_clusters=10*4, random_state=0).fit( X )
print(kmeans.labels_)

[ 8  3 31 ... 34 19 16]


In [5]:
def create_bag(labels_list):
    res=np.zeros((10*4,))
    for i in labels_list:
        res[i]+=1
    return res
#Как работает bag-of words? Допустим, уникальные элементы - 0,1,2,3,4.
#Тогда [0,2,0,2,4,4,4,3,3,0,1] мы посчитаем частоты каждого ->[3,1,2,2,3]. 
#То что получилось - это 5 фичей, на основе которых будем проводить классификацию.
list_of_dicts=[]
for i in labels:
    for j in filelists[i]:
        pathname=f'{i}/{j}'
        if(pathname in failed_files):
            continue
        des=des_dict[pathname]
        kmeans_labels=kmeans.predict(des)
        bag_of_words=create_bag(kmeans_labels)
        temp_dict={}
        temp_dict['pathname']=pathname
        for k in np.arange(10*4):
            temp_dict[f'feature_{k}']=bag_of_words[k]
        temp_dict['target']=i
        list_of_dicts.append(temp_dict)

df=pd.DataFrame(list_of_dicts)
df.head()
df.to_csv(f"{name_estimator}_dataset.csv",index=False)

In [6]:
from joblib import dump, load
dump(kmeans, f'written_models/{name_estimator}_kmeans.joblib') 

['written_models/sift_kmeans.joblib']